In [2]:
import pandas as pd
import numpy as np
import requests
import time
import csv
import json
from bs4 import BeautifulSoup

In [4]:
#https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=amzn&type=10-q&dateb=&owner=exclude&count=40
df = pd.read_csv('symbols_sectors.csv')
df.head()

,symbol,sector,industry,Name,Summary Quote
0,A,Healthcare,Diagnostics & Research,"Agilent Technologies, Inc.",http://www.nasdaq.com/symbol/a
1,AA,Basic Materials,Aluminum,Alcoa Inc.,http://www.nasdaq.com/symbol/aa
2,AAN,Industrials,Rental & Leasing Services,"Aaron&#39;s, Inc.",http://www.nasdaq.com/symbol/aan
3,AAON,Basic Materials,Building Materials,"AAON, Inc.",http://www.nasdaq.com/symbol/aaon
4,AAP,Consumer Cyclical,Specialty Retail,Advance Auto Parts Inc,http://www.nasdaq.com/symbol/aap


In [7]:
symbols = df['symbol'].unique()
symbols

array(['A', 'AA', 'AAN', ..., 'ZIXI', 'ZNGA', 'ZUMZ'], dtype=object)

In [8]:
url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=amzn&type=10-q&dateb=&owner=exclude&count=40"
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [9]:
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ENG">
 <head>
  <title>
   EDGAR Search Results
  </title>
  <link href="/include/interactive.css" rel="stylesheet" type="text/css"/>
  <link href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0001018724&amp;type=10-q%25&amp;dateb=&amp;owner=exclude&amp;count=40&amp;output=atom" rel="alternate" title="ATOM" type="application/atom+xml"/>
 </head>
 <body style="margin: 0">
  <!-- SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo -->
  <noscript>
   <iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0">
   </iframe>
  </noscript>
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagm

In [29]:
import re
results = soup.find('p', class_="identInfo")
#result = results.find_all('br')
results

<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=5961&amp;owner=exclude&amp;count=40">5961</a> - RETAIL-CATALOG &amp; MAIL-ORDER HOUSES<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=WA&amp;owner=exclude&amp;count=40">WA</a> | State of Inc.: <strong>DE</strong> | Fiscal Year End: 1231<br/>(Office of Trade &amp; Services)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0001018724"><b>insider transactions</b></a> for this <b>issuer</b>.
<br/>Get <a href="/cgi-bin/own-disp?action=getowner&amp;CIK=0001018724"><b>insider transactions</b></a> for this <b>reporting owner</b>.
</p>

In [33]:
res = str(results)
res

'<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=5961&amp;owner=exclude&amp;count=40">5961</a> - RETAIL-CATALOG &amp; MAIL-ORDER HOUSES<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=WA&amp;owner=exclude&amp;count=40">WA</a> | State of Inc.: <strong>DE</strong> | Fiscal Year End: 1231<br/>(Office of Trade &amp; Services)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0001018724"><b>insider transactions</b></a> for this <b>issuer</b>.\n<br/>Get <a href="/cgi-bin/own-disp?action=getowner&amp;CIK=0001018724"><b>insider transactions</b></a> for this <b>reporting owner</b>.\n</p>'

In [34]:
matches = re.findall('\d+',res)
matches

['5961', '40', '5961', '40', '1231', '0001018724', '0001018724']

In [35]:
matches[-1]

'0001018724'

In [36]:
matches[-3]

'1231'

In [37]:
results = soup.find_all('table', class_="tableFile2")
results

[<table class="tableFile2" summary="Results">
 <tr>
 <th scope="col" width="7%">Filings</th>
 <th scope="col" width="10%">Format</th>
 <th scope="col">Description</th>
 <th scope="col" width="10%">Filing Date</th>
 <th scope="col" width="15%">File/Film Number</th>
 </tr>
 <tr>
 <td nowrap="nowrap">10-Q</td>
 <td nowrap="nowrap"><a href="/Archives/edgar/data/1018724/000101872419000089/0001018724-19-000089-index.htm" id="documentsbutton"> Documents</a>  <a href="/cgi-bin/viewer?action=view&amp;cik=1018724&amp;accession_number=0001018724-19-000089&amp;xbrl_type=v" id="interactiveDataBtn"> Interactive Data</a></td>
 <td class="small">Quarterly report [Sections 13 or 15(d)]<br/>Acc-no: 0001018724-19-000089 (34 Act)  Size: 7 MB            </td>
 <td>2019-10-25</td>
 <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=000-22513&amp;owner=exclude&amp;count=40">000-22513</a><br/>191167298         </td>
 </tr>
 <tr class="blueRow">
 <td nowrap="nowrap">10-Q</td>
 <td

In [39]:
cik_qtr = []
for symbol in symbols:
    url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="+symbol+"&type=10-q&dateb=&owner=exclude&count=40"
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    results = soup.find('p', class_="identInfo")
    cik_qtr.append(symbol)
    if (results):
        res = str(results)
        matches = re.findall('\d+',res)
        cik_list = []
    
        cik = matches[-1]
        qtr_end = matches[-3]
        
        cik_list.append(cik)
        cik_list.append(qtr_end)
        cik_qtr.append(cik_list)
    else:
        cik_list.append('NULL')
        cik_list.append('NULL')
        cik_qtr.append(cik_list)
    print(results)

    

<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=3826&amp;owner=exclude&amp;count=40">3826</a> - LABORATORY ANALYTICAL INSTRUMENTS<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=CA&amp;owner=exclude&amp;count=40">CA</a> | State of Inc.: <strong>DE</strong> | Fiscal Year End: 1031<br/>formerly: AGILENT TECHNOLOGIES INC (filings through 2019-09-16)<br/>(Office of Life Sciences)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0001090872"><b>insider transactions</b></a> for this <b>issuer</b>.
<br/>Get <a href="/cgi-bin/own-disp?action=getowner&amp;CIK=0001090872"><b>insider transactions</b></a> for this <b>reporting owner</b>.
</p>
<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=3334&amp;owner=exclude&amp;count=40">3334</a> - PRIMARY PRODUCTION OF ALUMINUM<br/>State location:

<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=7340&amp;owner=exclude&amp;count=40">7340</a> - SERVICES-TO DWELLINGS &amp; OTHER BUILDINGS<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=NY&amp;owner=exclude&amp;count=40">NY</a> | State of Inc.: <strong>DE</strong> | Fiscal Year End: 1031<br/>formerly: ABM INDUSTRIES INC (filings through 1995-01-27)<br/>formerly: AMERICAN BUILDING MAINTENANCE INDUSTRIES INC (filings through 1994-03-30)<br/>(Office of Trade &amp; Services)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0000771497"><b>insider transactions</b></a> for this <b>issuer</b>.
</p>
<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=3841&amp;owner=exclude&amp;count=40">3841</a> - SURGICAL &amp; MEDICAL INSTRUMENTS &amp; APPARATUS<br/>State location: <a href="/cgi-bin

<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=7389&amp;owner=exclude&amp;count=40">7389</a> - SERVICES-BUSINESS SERVICES, NEC<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=L2&amp;owner=exclude&amp;count=40">L2</a> | State of Inc.: <strong>L2</strong> | Fiscal Year End: 0831<br/>(Office of Trade &amp; Services)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0001467373"><b>insider transactions</b></a> for this <b>issuer</b>.
</p>
<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=2836&amp;owner=exclude&amp;count=40">2836</a> - BIOLOGICAL PRODUCTS (NO DIAGNOSTIC SUBSTANCES)<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=NY&amp;owner=exclude&amp;count=40">NY</a> | Fiscal Year End: 1231<br/>(Office of Life Sciences)<br/>Get <a href="/cgi-bin/ow

<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=7372&amp;owner=exclude&amp;count=40">7372</a> - SERVICES-PREPACKAGED SOFTWARE<br/>State location: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;State=CA&amp;owner=exclude&amp;count=40">CA</a> | State of Inc.: <strong>DE</strong> | Fiscal Year End: 0131<br/>formerly: AUTODESK INC (filings through 2019-09-25)<br/>(Office of Technology)<br/>Get <a href="/cgi-bin/own-disp?action=getissuer&amp;CIK=0000769397"><b>insider transactions</b></a> for this <b>issuer</b>.
<br/>Get <a href="/cgi-bin/own-disp?action=getowner&amp;CIK=0000769397"><b>insider transactions</b></a> for this <b>reporting owner</b>.
</p>
<p class="identInfo"><acronym title="Standard Industrial Code">SIC</acronym>: <a href="/cgi-bin/browse-edgar?action=getcompany&amp;SIC=3661&amp;owner=exclude&amp;count=40">3661</a> - TELEPHONE &amp; TELEGRAPH APPARATUS<br/>State location: <a href="/cgi

ConnectionError: HTTPSConnectionPool(host='www.sec.gov', port=443): Max retries exceeded with url: /cgi-bin/browse-edgar?action=getcompany&CIK=AEL&type=10-q&dateb=&owner=exclude&count=40 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x125706cf8>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [40]:
url

'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=AEL&type=10-q&dateb=&owner=exclude&count=40'

In [55]:
results = soup.find('table', class_="tableFile2")
tds = results.find_all('td')
tds[1].find('a')['href']

'/Archives/edgar/data/927003/000155837019007077/0001558370-19-007077-index.htm'

In [58]:
links = tds[1].find_all('a')
links[1]['href']

'/cgi-bin/viewer?action=view&cik=927003&accession_number=0001558370-19-007077&xbrl_type=v'

In [59]:
url = 'https://www.sec.gov'+links[1]['href']
url

'https://www.sec.gov/cgi-bin/viewer?action=view&cik=927003&accession_number=0001558370-19-007077&xbrl_type=v'

In [60]:
url = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=amzn&owner=exclude&action=getcompany&Find=Search'
response = requests.get(url)
soup = BeautifulSoup(response.text)
results = soup.find('table', class_="tableFile2")
results

<table class="tableFile2" summary="Results">
<tr>
<th scope="col" width="7%">Filings</th>
<th scope="col" width="10%">Format</th>
<th scope="col">Description</th>
<th scope="col" width="10%">Filed/Effective</th>
<th scope="col" width="15%">File/Film Number</th>
</tr>
<tr>
<td nowrap="nowrap">IRANNOTICE</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1018724/000101872419000091/0001018724-19-000091-index.htm" id="documentsbutton"> Documents</a></td>
<td class="small">Notice of disclosure filed pursuant to Section 219 of the Iran Threat Reduction and Syria Human Rights Act of 2012 (Exchange Act Section 13(r)).<br/>Acc-no: 0001018724-19-000091 (34 Act)  Size: 124 KB            </td>
<td>2019-10-25</td>
<td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=000-22513&amp;owner=exclude&amp;count=40">000-22513</a><br/>191167317         </td>
</tr>
<tr class="blueRow">
<td nowrap="nowrap">10-Q</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/1018724/00010

In [63]:
trs = results.find_all('tr')

In [133]:
url = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=amzn&owner=exclude&action=getcompany&Find=Search'
response = requests.get(url)
soup = BeautifulSoup(response.text)
results = soup.find('table', class_="tableFile2")
trs = results.find_all('tr')
trs[2].find('td').text

'10-Q'

In [140]:
qtr_t = []
dates = []
links = []
for tr in trs:
    line = tr.find('td')
    if line:
        if line.text == '10-Q'or line.text == '10-K':
            qtr_t.append(line.text)
            dates.append(tr.find_all('td')[3])
            links.append('https://www.sec.gov'+tr.find_all('a')[1]['href'])

qtr_t[0]
dates[0]
links[0]

TypeError: can only concatenate str (not "NoneType") to str

In [136]:
trs[1].find_all('a')

[<a href="/Archives/edgar/data/1018724/000101872419000091/0001018724-19-000091-index.htm" id="documentsbutton"> Documents</a>,
 <a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=000-22513&amp;owner=exclude&amp;count=40">000-22513</a>]